# Preprocesses infrax part of the data

The appartement part of this dataset has a missing hour due to summer winter time  
The other parts are in UTC

# Imports

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from pathlib import Path
import datetime
import tqdm
import pyxlsb
alt.data_transformers.disable_max_rows()

In [ ]:
LOLA = False
if LOLA: 
    DATA_PATH = Path('/Users/lolabotman/PycharmProjects/FluviusFullData/profiles') #Path Lola
    PREPROCESSED_PATH = Path('/Users/lolabotman/PycharmProjects/FluviusFullData/profiles/preprocessed/infrax')#Path Lola
else: 
    # PATH to the profile directory in the fluvius data
    DATA_PATH = Path('/cw/dtaiproj/ml/2020-FLAIR-VITO/Data-2020-11/FluviusData/profiles')
    # PATH to where the preprocessed files should be appear
    PREPROCESSED_PATH = Path('/cw/dtaiproj/ml/2020-FLAIR-VITO/profile-clustering/new_preprocessed/infrax')

PREPROCESSED_PATH.mkdir(mode = 0o770, parents = True, exist_ok=True)

# Parse code
This is simply all the code to parse every kind of dataset (not so clean I know)  

In [ ]:
# Converting a value to a datetime format  
def to_timestamp(index): 
#     return index.apply(lambda value: pyxlsb.convert_date(value) if not np.isnan(value) else value).round('1min')
    return [pd.to_datetime(pyxlsb.convert_date(value)).round('1min') if not np.isnan(value) else value for value in index]

# transform the data and save the transformed data using the functions according to the parse data dict
def transform_and_save(source_path, name, parse_function): 
    info_path = PREPROCESSED_PATH / f"{name}_info.csv"
    data_path = PREPROCESSED_PATH / f"{name}_data.csv"
    if not( info_path.exists() and data_path.exists()):
        try:
            info_df, data_df = parse_function(source_path)
            info_df.to_csv(info_path)
            data_df.to_csv(data_path)
            print(data_df.shape)
            assert info_path.exists() 
            assert data_path.exists()
        except Exception as e:
            raise e
    else: 
        print(f'{name} already preprocessed')

# In these files, the 24 first rows are meta data and the time series starts on row 26
# There are 30 EANs and hourly measurements for a year (2015 file and 2014 file)
# Hypothesis : measurement values are in kW
def read_infrax_gas(path): 
    df = pd.read_excel(path, header = None, parse_dates = True)
    df = df.set_index(df.columns[0])
    smart_meter_df = df.T
    smart_meter_df = smart_meter_df.set_index('EAN_coded')
    
    # info_df (we get the meta data from the first 24 columns - table has been transposed)
    info_df = smart_meter_df.iloc[:,:24]
    info_df = info_df.loc[:, ~ info_df.columns.isna()]
    info_df = info_df.dropna(how='all', axis = 1) # drop columns with all NaN's
    info_df = info_df.set_index('Jaar', append=True)
    
    #data_df (we get the actual value data from after column 24 )
    data_df = smart_meter_df.iloc[:,24:].copy()
    data_df.columns = pd.to_datetime(data_df.columns).round('1min')
    data_df['Jaar'] = info_df.Jaar
    data_df = data_df.set_index(['Jaar'], append = True)
    data_df = data_df.sort_index()
    
    assert info_df.index.is_unique, 'info_df index should be unique'
    assert data_df.index.is_unique, 'data_df index should be unique'

    # sort on index
    info_df = info_df.sort_index()
    data_df = data_df.sort_index()
    
    
    return info_df, data_df 

# In these files, the 24 first rows are meta data and the time series starts on row 27
# there are 8 smart meter ids (EAN)
# yearly total is in kWh 
# measurement values are in kW
def read_infrax_heatpump(path): 
    df = pd.read_excel(path, header = None, parse_dates = True)
    df = df.set_index(df.columns[0])
    smart_meter_df = df.T
    smart_meter_df = smart_meter_df.set_index('EAN_coded')

    # info_df 
    info_df = smart_meter_df.iloc[:,:25]
    info_df = info_df.loc[:, ~ info_df.columns.isna()]
    info_df = info_df.dropna(how='all', axis = 1) # drop columns with all NaN's
    info_df = info_df.set_index('Jaar', append=True)
    
    #data_df 
    data_df = smart_meter_df.iloc[:,25:].copy()
#     print("doing a possibly WRONG conversion to date time")
    data_df.columns = pd.to_datetime(data_df.columns, format = "%d/%b %H:%M").round('1min')
    data_df['Jaar'] = info_df.index.get_level_values('Jaar')
    data_df = data_df.set_index(['Jaar'], append = True)
    data_df = data_df.sort_index()
    
    assert info_df.index.is_unique, 'info_df index should be unique'
    assert data_df.index.is_unique, 'data_df index should be unique'

    # sort on index
    info_df = info_df.sort_index()
    data_df = data_df.sort_index()
    
    return info_df, data_df 

# Yearly total in kwh 
# hyp : Measurement values in kw >> deduced from the fact that (sum of the measurement) = 4*(jaarverbuik in kwh) + max value given in kW
def read_infrax_app_xlsb(path): 
    # no useful index! 
    df = pd.read_excel(path, header = None, engine='pyxlsb')
    df.set_index(df.columns[0], inplace = True)
    smart_meter_df = df.T
   
    #info df 
    info_df = (
    smart_meter_df
        .iloc[:,:5] # info columns
        .loc[:,~smart_meter_df.columns[:5].isna()] # drop nan columns
    )
    info_df = info_df.rename(index=lambda s: 'app2_'+ str(s)) #changing to 'unique' index to not mix up with the app1
    info_df['Jaar']=[2014]*len(info_df) ##extract 2014 anoher way ? more generic ?
    info_df = info_df.set_index('Jaar', append=True)
    info_df = info_df.dropna(how = 'all') # for some reason there are some NaN rows
    info_df = info_df.rename(columns={'Max (kW)':'Piek P (kW)'}) #rename such as to have the same column title as the other dfs
    
    # data_df
    data_df = smart_meter_df.iloc[:,5:].copy()
    data_df.columns = to_timestamp(data_df.columns)
    data_df = data_df.dropna(how = 'all') # for some reason there are some NaN rows
    data_df.columns = data_df.columns.round('1min')
    data_df = data_df.rename(index=lambda s: 'app2_'+ str(s))
    data_df['Jaar'] = [int(2014)]*len(data_df)
    data_df = data_df.set_index(['Jaar'], append = True)  
    data_df = data_df.loc[:,pd.to_datetime(data_df.columns).year == 2014] #there is one day of 2015 that we don't want to keep
    data_df.columns = data_df.columns.map(lambda t: t.replace(year=2016)) #set the columns to 2016 for the final merge 
    data_df.columns = data_df.columns.round('1min')
    data_df = data_df.astype('float')
    
    # duplicate hour take the mean
    data_df = data_df.groupby(by=data_df.columns, axis = 1).mean()
    
    assert info_df.index.is_unique, 'info_df index should be unique'
    assert data_df.index.is_unique, 'data_df index should be unique'
    assert info_df.columns.is_unique, 'info_df columns should be unique'
    assert data_df.columns.is_unique, 'data_df columns should be unique'

    # sort on index
    info_df = info_df.sort_index()
    data_df = data_df.sort_index()
    return info_df, data_df

# hyp : all measurement values in kW >> deduced from the fact that (sum of the measurement) = 4*(jaarverbuik in kwh) 
# jaar verbruik in kWh
def read_infrax_app_xlsx(path): 
    df = pd.read_excel(path, header = None, parse_dates = True)
    df = df.set_index(df.columns[0])
    smart_meter_df = df.T

    # info_df
    info_df = smart_meter_df.iloc[:,:7]
    info_df = info_df.loc[:, ~ info_df.columns.isna()]
    info_df = info_df.rename(index=lambda s: 'app1_'+ str(s)) #changing to 'unique' index to not mix up with the app1
    info_df['Jaar']=[int(2014)]*len(info_df) ##extract 2014 anoher way ? more generic ?
    info_df = info_df.set_index('Jaar', append=True)
    info_df = info_df.rename(columns={'Max (kW)':'Piek P (kW)'})#rename such as to have the same column title as the other dfs
    
    # data_df
    data_df = smart_meter_df.iloc[:,7:].copy()
    data_df = data_df.dropna(how = 'all')
    data_df.columns = pd.to_datetime(data_df.columns, format = '%m/%d/%Y %H:%M').round('1min')
    data_df = data_df.rename(index=lambda s: 'app1_'+ str(s))
    data_df['Jaar'] = [2014]*len(data_df)
    data_df = data_df.set_index(['Jaar'], append = True)
    data_df = data_df.loc[:,pd.to_datetime(data_df.columns).year == 2014] #there is one day of 2015 that we don't want to keep
    data_df.columns = data_df.columns.map(lambda t: t.replace(year=2016)) #set the columns to 2016 for the final merge 
    data_df = data_df.astype('float')
    
    # duplicate hour take the mean
    data_df = data_df.groupby(by=data_df.columns, axis = 1).mean()
    
    assert info_df.index.is_unique, 'info_df index should be unique'
    assert data_df.index.is_unique, 'data_df index should be unique'
    assert info_df.columns.is_unique, 'info_df columns should be unique'
    assert data_df.columns.is_unique, 'data_df columns should be unique'

    # sort on index
    info_df = info_df.sort_index()
    data_df = data_df.sort_index()
    return info_df, data_df

# all other files ending in _coded.xlsb 
# measurment values in kW
# yearly total in kWh
def read_infrax_data(path):
    df = pd.read_excel(path, engine='pyxlsb')
    df.set_index(df.columns[0], inplace = True)
    smart_meter_df = df.T
    smart_meter_df.set_index('EAN_coded', inplace = True)


    # info df 
    info_df = (
    smart_meter_df
        .iloc[:,:21] # info columns
        .loc[:,~smart_meter_df.columns[:21].isna()] # drop nan columns
        .drop(columns = ['Info installatie', 'Info profiel'])
    )

    info_df['PV vermogen (kW)'] = info_df['PV vermogen (kW)'].replace('/', np.nan)
    info_df = info_df[~ info_df.index.isna()] #remove row with nan index
    info_df = info_df.reset_index()
    info_df['EAN_coded'] = info_df['EAN_coded'].astype('int')
    info_df = info_df.set_index(['EAN_coded', 'Jaar'])

    # data df 

    data_df = smart_meter_df.iloc[:,23:].copy()
    data_df.columns = to_timestamp(data_df.columns)
    # drop the columns with NaT
    data_df = data_df.loc[:,~data_df.columns.isna()]
    data_df.columns = data_df.columns.round('1min')
    data_df = data_df[~ data_df.index.isna()] #remove row with nan index
    data_df = data_df.reset_index()
    data_df['EAN_coded'] = data_df['EAN_coded'].astype('int')
    data_df['Jaar'] = info_df.index.get_level_values(1)
    data_df = data_df.set_index(['EAN_coded','Jaar'])

    # Handle the ids 1290 en 1299 that have year 2013 twice 
    if (1290, 2013) in info_df.index: 
        new_info_df = info_df.reset_index()
        new_info_df.loc[new_info_df['EAN_coded'].isin([1290,1299]) & new_info_df.duplicated(subset = ['EAN_coded', 'Jaar'], keep = 'first'), 'Jaar'] = 2012
        info_df = new_info_df.set_index(['EAN_coded', 'Jaar'])
        
        new_data_df = data_df.reset_index()
        new_data_df.loc[new_data_df['EAN_coded'].isin([1290,1299]) & new_data_df.duplicated(subset = ['EAN_coded', 'Jaar'], keep = 'first'), 'Jaar'] = 2012
        data_df = new_data_df.set_index(['EAN_coded', 'Jaar'])
        
   
    assert info_df.index.is_unique, 'info_df index should be unique'
    assert data_df.index.is_unique, 'data_df index should be unique'

    # sort on index
    info_df = info_df.sort_index()
    data_df = data_df.sort_index()
    
    return info_df, data_df 
    
  

# file information

In [ ]:
# this is which parser function to use for which file 
parser_functions = {   
    'Appartement1': read_infrax_app_xlsx,
    'Appartement2': read_infrax_app_xlsb,
#     'SLP_profiel S41 2014 (30)_coded': read_infrax_gas,
#     'SLP_profiel S41 2015 (30)_coded': read_infrax_gas,
    'SLPs_professionelen(348)_coded': read_infrax_data,
    'SLPs_residentielen(1675)_coded': read_infrax_data,
    'SLPs_residentiëlen(1675)_coded': read_infrax_data,
    'Slimme meters met WP (en eventueel PV)_coded': read_infrax_heatpump,
    'Slimme meters_professionelen(141)_coded': read_infrax_data,
    'Slimme meters_prosumers(123)_coded': read_infrax_data,
    'Slimme meters_residentielen(1080)_coded': read_infrax_data, 
    'Slimme meters_residentiëlen(1080)_coded': read_infrax_data
}

# this is which preprocessed file name to use 
new_filename = { 
    'Appartement1': 'app1',
    'Appartement2': 'app2',
    'SLP_profiel S41 2014 (30)_coded': 'SLP_gas_2014',
    'SLP_profiel S41 2015 (30)_coded': 'SLP_gas_2015',
    'SLPs_professionelen(348)_coded': 'SLP_prof',
    'SLPs_residentielen(1675)_coded': 'SLP_resid',
    'SLPs_residentiëlen(1675)_coded': 'SLP_resid',
    'Slimme meters met WP (en eventueel PV)_coded': 'M_heatpump',
    'Slimme meters_professionelen(141)_coded': 'M_prof',
    'Slimme meters_prosumers(123)_coded': 'M_prosumers',
    'Slimme meters_residentielen(1080)_coded': 'M_resid', 
    'Slimme meters_residentiëlen(1080)_coded': 'M_resid'
}

# Parse it all :D 

In [ ]:
infrax_path = DATA_PATH/ "20171219 Profielen Infrax"
translate = dict()
for path in tqdm.tqdm(list(infrax_path.glob('**/*.xlsb'))+ list(infrax_path.glob('**/*.xlsx'))):
    print(path)
    if path.stem in parser_functions:
        new_name = new_filename[path.stem]
        parser = parser_functions[path.stem]
        transform_and_save(path, new_name, parser)
    else:
        print('error:'+path.stem)
   
    



# Make combined dataframe of relevant profiles
So these profiles are all in the same format so we can easily combine these!  
I add some extra columns to the info dataframe to ensure that we can later recover the different groups if necessary.  
Appartement is excluded and the gas information is excluded

In [ ]:
def combined_info_df():
    files = ['M_resid_info.csv', 'SLP_resid_info.csv', 'M_prof_info.csv', 'SLP_prof_info.csv', 'M_prosumers_info.csv', 'M_heatpump_info.csv', 'app1_info.csv', 'app2_info.csv']
    files = [PREPROCESSED_PATH/file for file in files]


    M_heatpump = pd.read_csv(files[5], index_col = [0,1])
    M_heatpump['heatpump'] = True


    M_prosumers = pd.read_csv(files[4], index_col = [0,1])
    M_prosumers = M_prosumers.dropna(how='all')
    M_prosumers['prosumer'] = True

    
    M_prof_df = pd.read_csv(files[2], index_col = [0,1])
    SLP_prof_df = pd.read_csv(files[3], index_col = [0,1])


    M_resid_df = pd.read_csv(files[0], index_col = [0,1])
    SLP_resid_df = pd.read_csv(files[1], index_col = [0,1])
    
    app1_df = pd.read_csv(files[6], index_col =[0,1])
    app1_df['R/P']=['app1']*len(app1_df.index)
    
    app2_df = pd.read_csv(files[7], index_col = [0,1])
    app2_df['R/P']=['app2']*len(app2_df.index)

    infrax = pd.concat([M_resid_df, SLP_resid_df, M_prof_df, SLP_prof_df, M_heatpump, M_prosumers, app1_df, app2_df]).sort_index()
    infrax.to_csv(PREPROCESSED_PATH/'combined_info.csv')

In [ ]:
OVERWRITE = True
if not (PREPROCESSED_PATH/'combined_info.csv').exists() or OVERWRITE: 
    combined_info_df()

In [ ]:
comb = pd.read_csv(PREPROCESSED_PATH/'combined_info.csv')
comb

In [ ]:
OVERWRITE = True
if not (PREPROCESSED_PATH/'combined_data.csv').exists() or OVERWRITE: 
    files = ['M_resid_info.csv', 'SLP_resid_info.csv', 'M_prof_info.csv', 'SLP_prof_info.csv', 'M_prosumers_info.csv', 'M_heatpump_info.csv','app1_info.csv', 'app2_info.csv']
    profile_files = [PREPROCESSED_PATH/ f'{file[:-8]}data.csv' for file in files]
    combined_data_df = pd.concat([pd.read_csv(file, index_col = [0,1]) for file in profile_files] )
    combined_data_df = combined_data_df.dropna(how='all', axis = 0).sort_index()
    combined_data_df = combined_data_df.reset_index()
    combined_data_df['Jaar'] = combined_data_df['Jaar'].astype('int')
    combined_data_df = combined_data_df.set_index(['EAN_coded','Jaar'])
    combined_data_df.to_csv(PREPROCESSED_PATH/'combined_data.csv')

In [ ]:
combined_data_df

We know that the 2 appartment files have not been treated, because we can still notice the artefacts (duplicate values in october and missing values in march) >> let's remove this before checking the rest

# Make clean info df

In [ ]:
info_df = pd.read_csv(PREPROCESSED_PATH/'combined_info.csv')
info_df

In [ ]:
clean_info_df = (
    info_df
    # remove two NaN rows
    .dropna(axis = 0, how = 'all', subset = ['EAN_coded'])
    # drop unnecessary columns 
    .drop(columns = ['DNB', 'Automaat', 'Bron SLP of SM?', 'Meting of synthetisch?', 'nacht/dag ratio', 'Piek P (kW)', 'Jaarverbruik (kWh)', 'Gebruiksduur'])
    # rename some columns
    .rename(columns = {'EAN_coded':'meterID', 'R/P':'consumer_type', 'Jaar':'year', 'Gemeente':'town', 'Postcode':'postal_code', 'SLP cat': 'SLP_cat','#gezinsleden':'#family_members', 'Aansluitvermogen (kVA)':'connection_power', 'PV?':'PV', 'PV vermogen (kW)':'PV_power'})
    .astype({'year':'int'})
    .set_index(['meterID', 'year'])
    .fillna({'prosumer': False, 'heatpump':False})
)

In [ ]:
clean_info_df['PV'] = clean_info_df['PV'].replace({'N':False, 'Y': True})
clean_info_df['consumer_type'] = clean_info_df['consumer_type'].replace({'R':'residential', 'P':'professional'})
clean_info_df['data_source'] = 'Infrax'
clean_info_df

# Change connection capacity from kVA to kWh
**TODO: to do this conversion we would need a power factor**: https://www.adeltd.co.uk/info/what-is-kva.php 


assumption 1kVa = 1kW (CONFIRMED by reinhilde)

# Change PV power from kW to kWh? 
Is this necessary? 

In [ ]:
clean_info_df.to_csv(PREPROCESSED_PATH/ 'clean_info.csv')

# Make clean data df 

In [ ]:
data_df = pd.read_csv(PREPROCESSED_PATH/'combined_data.csv')
# data_df

In [ ]:
clean_data_df = (
    data_df
    .rename(columns = {'EAN_coded': 'meterID', 'Jaar':'year'})
    .astype({'year': 'int'})
    .set_index(['meterID', 'year'])
)
clean_data_df

In [ ]:
clean_data_df.columns = pd.to_datetime(clean_data_df.columns)

# Check for summer/winter time issues

In [ ]:
clean_data_df.loc[:, '2016-3-29 1:00': '2016-3-29 4:00']

In [ ]:
clean_data_df.loc[:, '2016-10-10 1:00': '2016-10-10 4:00' ]

# Convert measurements from kW to kWh

In [ ]:
clean_data_df = clean_data_df / 4 

# Store the result

In [ ]:
clean_data_df.to_csv(PREPROCESSED_PATH/'clean_data.csv')

In [ ]:
clean_data_df.iloc[:,-100:]

# Investigate duplicates issue!

In [ ]:
%%time
path = Path('/cw/dtaiproj/ml/2020-FLAIR-VITO/Data-2020-11/FluviusData/profiles/20171219 Profielen Infrax/1. Consumenten/1. Residentiële profielen/SLPs_residentiëlen(1675)_coded.xlsb')
df = pd.read_excel(path, engine='pyxlsb')
df.set_index(df.columns[0], inplace = True)

smart_meter_df = df.T
smart_meter_df.set_index('EAN_coded', inplace = True)

smart_meter_df

# info df 
info_df = (
smart_meter_df
    .iloc[:,:21] # info columns
    .loc[:,~smart_meter_df.columns[:21].isna()] # drop nan columns
    .drop(columns = ['Info installatie', 'Info profiel'])
)
info_df['PV vermogen (kW)'] = info_df['PV vermogen (kW)'].replace('/', np.nan)
info_df = info_df[~ info_df.index.isna()] #remove row with nan index
info_df = info_df.reset_index()
info_df['EAN_coded'] = info_df['EAN_coded'].astype('int')
info_df = info_df.set_index(['EAN_coded', 'Jaar'])
info_df

### Indices that occur twice

In [ ]:
counts = info_df.index.value_counts()
duplicate_counts = counts[counts>1]
duplicate_counts

In [ ]:
duplicate_info_df = info_df.loc[duplicate_counts.index]
duplicate_info_df

In [ ]:
info_df.loc[[1290, 1299],:]

In [ ]:
info_df.index.get_level_values(1).unique()

### Look at corresponding profiles

In [ ]:
# data df 
data_df = smart_meter_df.iloc[:,23:].copy()
data_df.columns = to_timestamp(data_df.columns)
# drop the columns with NaT
data_df = data_df.loc[:,~data_df.columns.isna()]
data_df.columns = data_df.columns.round('1min')
data_df = data_df[~ data_df.index.isna()] #remove row with nan index
data_df = data_df.reset_index()
data_df['EAN_coded'] = data_df['EAN_coded'].astype('int')
data_df['Jaar'] = info_df.index.get_level_values(1)
data_df = data_df.set_index(['EAN_coded','Jaar'])

In [ ]:
duplicate_data_df = data_df.loc[duplicate_counts.index]
duplicate_data_df

temp_df = duplicate_data_df.reset_index().astype({'EAN_coded':'str'})
temp_df.loc[temp_df.duplicated(subset = ['EAN_coded', 'Jaar'], keep = 'last'), 'EAN_coded'] = temp_df.loc[temp_df.duplicated(subset = ['EAN_coded', 'Jaar'], keep = 'last'), 'EAN_coded'] + 'duplicate'
temp_df.set_index(['EAN_coded', 'Jaar'], inplace = True)
temp_df

In [ ]:
vis_df = temp_df.stack().to_frame('value').reset_index().rename(columns = {'level_2':'timestamp'})
vis_df.timestamp

alt.Chart(vis_df, width = 2200).mark_line().encode(
    x = 'timestamp:T',
    y  = 'value:Q', 
    color = 'EAN_coded:N'
).interactive(bind_y = False)